In [1]:
import pandas as pd
import os
import zipfile
import json
import re
from transformers import pipeline

c:\Users\justi\anaconda3\envs\capstone\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# base_dir = r"../data/TeleAntiFraud-28k/merged_result"
# rows = []

# for root, dirs, files in os.walk(base_dir):
#     for file in files:
#         if file == "config.json":
#             with open(os.path.join(root, file), "r") as f:
#                 data = json.load(f)
                
#             # Aggregate left/right messages
#             left_texts = [seg["content"] for seg in data["audio_segments"] if seg["role"] == "left"]
#             right_texts = [seg["content"] for seg in data["audio_segments"] if seg["role"] == "right"]

#             # Combine full conversation (in chronological order)
#             full_text = " ".join([seg["content"] for seg in data["audio_segments"]])

#             # Determine label (POS/NEG)
#             label = "POS" if "POS" in root else "NEG"

#             rows.append({
#                 "conversation_id": root.split(os.sep)[-1],
#                 "label": label,
#                 "left_text": " ".join(left_texts),
#                 "right_text": " ".join(right_texts),
#                 "full_text": full_text
#             })

# df = pd.DataFrame(rows)
# df.to_csv("../data/TeleAntiFraud/cleaned_conversations.csv", index=False, encoding="utf-8-sig")
# print(f"✅ Extracted {len(df)} conversations.")


✅ Extracted 9487 conversations.


In [ ]:
def detect_fraud_type(text):
    if pd.isna(text) or text.strip() == "":
        return None

    text = text.lower()

    # Define keyword patterns using chinese characters
    fraud_patterns = {
        "Investment Scam": r"(投资|理财|收益|基金|股票|回报|分红)",
        "Loan Scam": r"(贷款|借款|信用|征信|额度|分期)",
        "Telecom Scam": r"(电话|公安|银行|账户|冻结|验证码|转账)",
        "Impersonation Scam": r"(警察|客服|工作人员|冒充|身份)",
        "Lottery/Prize Scam": r"(中奖|礼品|领取|抽奖|奖励|活动)"
    }

    for fraud_type, pattern in fraud_patterns.items():
        if re.search(pattern, text):
            return fraud_type

    return "Other Scam"  # default for fraud but unknown type

In [17]:
df = pd.read_csv("../data/TeleAntiFraud/cleaned_conversations.csv")
df["fraud_type"] = df.apply(
    lambda row: detect_fraud_type(row["full_text"]) if row["label"] == "NEG" else None,
    axis=1
)

In [21]:
from deep_translator import GoogleTranslator # could also use other translators like deepl or openai, but google translator is the fastest
from tqdm import tqdm

translator = GoogleTranslator(source='zh-CN', target='en')

df["perp_text_en"] = ""
# df["victim_text_en"] = ""
# df["full_text_en"] = ""

for i in tqdm(range(len(df))):
    try:
        if df.loc[i, "label"] == "NEG": # could also use df.loc[i, "fraud_type"] == "Loan Scam" for just translating loan scam cases or comment out for all cases
            df.loc[i, "perp_text_en"] = translator.translate(df.loc[i, "left_text"]) if pd.notna(df.loc[i, "left_text"]) else ""
            # df.loc[i, "victim_text_en"] = translator.translate(df.loc[i, "right_text"]) if pd.notna(df.loc[i, "right_text"]) else ""
            # df.loc[i, "full_text_en"] = translator.translate(df.loc[i, "full_text"]) if pd.notna(df.loc[i, "full_text"]) else ""
    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue


 32%|███▏      | 3029/9487 [41:26<1:28:21,  1.22it/s]


KeyboardInterrupt: 

In [22]:
df.to_csv("../data/TeleAntiFraud/cleaned_conversations.csv", index=False, encoding="utf-8-sig")